In [1]:
import numpy as np
import pandas as pd

In [2]:
data1 = pd.read_csv('train.csv')

In [212]:
data1.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_69902,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Pink,0.80,7.78,13,9,0.0,1
2,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
3,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
4,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2


In [3]:
data1.isnull().mean()

pet_id            0.000000
issue_date        0.000000
listing_date      0.000000
condition         0.078418
color_type        0.000000
length(m)         0.000000
height(cm)        0.000000
X1                0.000000
X2                0.000000
breed_category    0.000000
pet_category      0.000000
dtype: float64

In [124]:
data = data1.drop(labels=["pet_id","issue_date","listing_date","breed_category","pet_category"],axis=1)

In [5]:
data.head()

,condition,color_type,length(m),height(cm),X1,X2
0,2.0,Brown Tabby,0.80,7.78,13,9
1,2.0,Pink,0.80,7.78,13,9
2,1.0,White,0.72,14.19,13,9
3,NaN,Brown,0.15,40.90,15,4
4,1.0,White,0.62,17.82,0,1


In [6]:
data_train = data[data["condition"].notnull()].drop("condition", axis=1)
data_test = data[data["condition"].isnull()].drop("condition", axis=1)

In [7]:
data_train.head()

,color_type,length(m),height(cm),X1,X2
0,Brown Tabby,0.80,7.78,13,9
1,Pink,0.80,7.78,13,9
2,White,0.72,14.19,13,9
4,White,0.62,17.82,0,1
5,Black,0.50,11.06,18,4


In [8]:
data_test.head()

,color_type,length(m),height(cm),X1,X2
3,Brown,0.15,40.90,15,4
22,Brown,0.45,14.00,11,4
36,White,0.03,21.90,13,9
39,Brown,0.73,8.22,0,6
44,Black,0.08,30.40,9,2


In [9]:
data_y = data[data["condition"].notnull()].condition

In [10]:
data_y.head()

0    2.0
1    2.0
2    1.0
4    1.0
5    2.0
Name: condition, dtype: float64

In [11]:
#encoding Categorical data - new way 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder',OneHotEncoder(sparse=False), [0])], remainder= 'passthrough')

In [12]:
ct.get_params 

<bound method ColumnTransformer.get_params of ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('encoder',
                                 OneHotEncoder(categories='auto', drop=None,
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='error',
                                               sparse=False),
                                 [0])],
                  verbose=False)>

In [13]:
ct.fit(data_train)

ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('encoder',
                                 OneHotEncoder(categories='auto', drop=None,
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='error',
                                               sparse=False),
                                 [0])],
                  verbose=False)

In [14]:
tmp = ct.transform(data_train)

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
classifier = RandomForestClassifier()

In [17]:
classifier.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)>

In [18]:
classifier.fit(tmp, data_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [19]:
pred_train = classifier.predict(tmp)

In [20]:
from sklearn.metrics import confusion_matrix
metrric = confusion_matrix(data_y, pred_train)

In [21]:
metrric

array([[6279,    1,    1],
       [   2, 6817,    0],
       [   0,    1, 4257]])

In [22]:
data_test.head()

,color_type,length(m),height(cm),X1,X2
3,Brown,0.15,40.90,15,4
22,Brown,0.45,14.00,11,4
36,White,0.03,21.90,13,9
39,Brown,0.73,8.22,0,6
44,Black,0.08,30.40,9,2


In [31]:
test_pred = classifier.predict(ct.transform(data_test)) 
test_pred

array([2., 2., 2., ..., 2., 0., 2.])

In [135]:
for index, row in data.iterrows():
  if math.isnan(row["condition"]):
    # print([[row["color_type"],row['length(m)'],row['height(cm)'], row['X1'], row['X2']]])
    # print(classifier.predict(ct.transform([[row["color_type"],row['length(m)'],row['height(cm)'], row['X1'], row['X2']]])))
    data.at[index, "condition"] = classifier.predict(ct.transform([[row["color_type"],row['length(m)'],row['height(cm)'], row['X1'], row['X2']]]))
    # print(row)
    

In [136]:
data_test = data[data["condition"].isnull()].drop("condition", axis=1)
data_test

,color_type,length(m),height(cm),X1,X2


In [152]:
data.shape

(18835, 6)

In [138]:
data.head()

,condition,color_type,length(m),height(cm),X1,X2
0,2.0,Brown Tabby,0.80,7.78,13,9
1,2.0,Pink,0.80,7.78,13,9
2,1.0,White,0.72,14.19,13,9
3,2.0,Brown,0.15,40.90,15,4
4,1.0,White,0.62,17.82,0,1


In [142]:
data.isnull().mean()

condition     0.0
color_type    0.0
length(m)     0.0
height(cm)    0.0
X1            0.0
X2            0.0
dtype: float64

In [143]:
test = pd.read_csv("test.csv")

In [144]:
test.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0.87,42.73,0,7
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,6.71,0,1
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,Black,0.24,41.21,0,7
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,Black,0.29,8.46,7,1
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,Brown,0.71,30.92,0,7


In [146]:
test1 = test.drop(labels=["pet_id","issue_date","listing_date"], axis=1)

In [147]:
test1.head()

,condition,color_type,length(m),height(cm),X1,X2
0,0.0,Black,0.87,42.73,0,7
1,1.0,Orange Tabby,0.06,6.71,0,1
2,1.0,Black,0.24,41.21,0,7
3,1.0,Black,0.29,8.46,7,1
4,1.0,Brown,0.71,30.92,0,7


In [149]:
test1.isnull().mean()

condition     0.076685
color_type    0.000000
length(m)     0.000000
height(cm)    0.000000
X1            0.000000
X2            0.000000
dtype: float64

In [150]:
for index, row in test1.iterrows():
  if math.isnan(row["condition"]):
    test1.at[index, "condition"] = classifier.predict(ct.transform([[row["color_type"],row['length(m)'],row['height(cm)'], row['X1'], row['X2']]]))    

In [151]:
test1.isnull().mean()

condition     0.0
color_type    0.0
length(m)     0.0
height(cm)    0.0
X1            0.0
X2            0.0
dtype: float64

In [155]:
Train_X = data
Train_Y = data1[['breed_category','pet_category']]
Train_X.head()

,condition,color_type,length(m),height(cm),X1,X2
0,2.0,Brown Tabby,0.80,7.78,13,9
1,2.0,Pink,0.80,7.78,13,9
2,1.0,White,0.72,14.19,13,9
3,2.0,Brown,0.15,40.90,15,4
4,1.0,White,0.62,17.82,0,1


In [157]:
 ct2 = ColumnTransformer(transformers = [('encoder',OneHotEncoder(sparse=False), [1])], remainder= 'passthrough')
 ct2.fit(Train_X)

ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('encoder',
                                 OneHotEncoder(categories='auto', drop=None,
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='error',
                                               sparse=False),
                                 [1])],
                  verbose=False)

In [158]:
classifier2 = RandomForestClassifier()
classifier2.fit(ct2.transform(Train_X),Train_Y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [161]:
T_pred=classifier2.predict(ct2.transform(test1))

In [276]:
f = open('result.csv',"w")
count=0
f.write('pet_id,breed_category,pet_category')
for i in range(len(T_pred)):
    f.write(f'\n{test["pet_id"][i]},{int(Test_breed[i])},{int(Test_cat[i])} ')
    count+=1
f.close()

In [171]:
 from sklearn.ensemble import GradientBoostingClassifier

In [268]:
classifier3 =  GradientBoostingClassifier(n_estimators=220,verbose=1,learning_rate=0.5)
classifier3.fit(ct2.transform(Train_X),Train_Y.breed_category)

      Iter       Train Loss   Remaining Time 
         1        9477.7882           24.71s
         2        7501.5222           26.41s
         3        6593.3079           26.50s
         4        6074.5481           26.72s
         5        5819.0168           26.83s
         6        5672.5341           26.93s
         7        5570.0320           26.94s
         8        5539.5563           27.69s
         9        5508.4632           27.68s
        10        5457.9586           27.47s
        11        5426.6552           27.24s
        12        5377.8079           27.09s
        13        5355.9590           26.99s
        14        5335.6193           26.97s
        15        5318.0167           26.83s
        16        5310.3794           26.77s
        17        5295.5285           26.62s
        18        5268.3102           26.46s
        19        5251.4448           26.25s
        20        5229.4786           26.07s
        21        5212.9073           26.09s
        2

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.5, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=220,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=2,
                           warm_start=False)

In [244]:
classifier4 =  GradientBoostingClassifier(n_estimators=100,verbose=1,learning_rate=0.2)
classifier4.fit(ct2.transform(Train_X),Train_Y.pet_category)

      Iter       Train Loss   Remaining Time 
         1       13422.7188           17.15s
         2       12307.7230           17.72s
         3       11550.2437           17.21s
         4       10938.9046           17.21s
         5       10471.5023           16.92s
         6       10083.7340           16.78s
         7        9759.7302           16.50s
         8        9442.3075           16.33s
         9        9131.9604           16.28s
        10        8928.8911           16.11s
        20        7620.0593           14.01s
        30        6968.7455           12.00s
        40        6591.3272           10.19s
        50        6316.9454            8.49s
        60        6076.2413            6.77s
        70        5905.9538            5.07s
        80        5772.2508            3.38s
        90        5658.3456            1.68s
       100        5543.0957            0.00s


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.2, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=1,
                           warm_start=False)

In [176]:
Train_Y.columns

Index(['breed_category', 'pet_category'], dtype='object')

In [177]:
Train_Y.breed_category

0        0.0
1        0.0
2        0.0
3        2.0
4        0.0
        ... 
18830    0.0
18831    2.0
18832    1.0
18833    1.0
18834    1.0
Name: breed_category, Length: 18835, dtype: float64

In [274]:
Test_breed = classifier3.predict(ct2.transform(test1))
Test_cat = classifier4.predict(ct2.transform(test1))

In [269]:
Train_breed = classifier3.predict(ct2.transform(Train_X))
Train_cat = classifier4.predict(ct2.transform(Train_X))

In [272]:
met1 = confusion_matrix(Train_Y.breed_category,Train_breed)
met2 = confusion_matrix(Train_Y.pet_category,Train_cat)

In [273]:
met1

array([[8675,  288,   38],
       [ 456, 7866,   35],
       [ 244,  192, 1041]])

In [189]:
met2

array([[   19,     8,    50,    11],
       [    1,  5688,  1485,    11],
       [    1,   547, 10032,    41],
       [    0,    56,   196,   689]])

In [194]:
from sklearn.metrics import accuracy_score

In [270]:
acc = accuracy_score(Train_Y.breed_category,Train_breed)
acc1 = accuracy_score(Train_Y.pet_category,Train_cat)

In [271]:
acc

0.9334749137244491

In [251]:
acc1

0.8784178391292806